In [1]:
#Introduire les données de stock
!pip install yfinance
!pip install matplotlib
import yfinance as yf
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#Définir la liste de stocks
stock_symbols=['AAPL','AMZN','TSLA','GOOGL','PFE']

#Spécifier le chemin du dossier de données
data_folder='Data'

#Si le dossier n'existe pas, créez-le
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

#Obtenez les données historiques de chaque ticker et enregistrez-les dans un fichier CSV
for symbol in stock_symbols:
    #Chemin du fichier de données de construction
    file_path=os.path.join(data_folder,f'{symbol}.csv')

    #Vérifiez si le fichier de données existe déjà
    if os.path.exists(file_path):
        print(f'{file_path} already exists,skipping...')
        continue

    #Télécharger les données
    print(f'Downloading{symbol}data...')
    data=yf.download(symbol,start='2022-01-01',end='2023-12-31')

    #Enregistrer les données dans un fichier CSV
    data.to_csv(file_path)
    print(f'Data saved to {file_path}')

    #Lire des données
    data=pd.read_csv(file_path,index_col=0,parse_dates=True)

    #print les données de l'historique des stocks
    print(f'\n{symbol}Historical Data:')
    print(data)
    print('\n' + '='*50 + '\n')

print('Data download completed.')


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Data\AAPL.csv already exists,skipping...
Data\AMZN.csv already exists,skipping...
Data\TSLA.csv already exists,skipping...
Data\GOOGL.csv already exists,skipping...
Data\PFE.csv already exists,skipping...
Data download completed.


In [2]:
AAPL=yf.Ticker("AAPL")
AAPL_cours=AAPL.history(start='2022-01-01',end='2023-12-31')#affecter des données historiques de AAPL à AAPL_cours
AAPL_cours

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-01-03 00:00:00-05:00,175.597043,180.583634,175.478554,179.724548,104487900,0.0,0.0
2022-01-04 00:00:00-05:00,180.336790,180.642895,176.870854,177.443573,99310400,0.0,0.0
2022-01-05 00:00:00-05:00,177.354683,177.907649,172.447089,172.723572,94537600,0.0,0.0
2022-01-06 00:00:00-05:00,170.531463,173.098822,169.484776,169.840256,96904000,0.0,0.0
2022-01-07 00:00:00-05:00,170.719093,171.953398,168.882448,170.008133,86709100,0.0,0.0
...,...,...,...,...,...,...,...
2023-12-22 00:00:00-05:00,194.931260,195.160978,192.724085,193.353287,37122800,0.0,0.0
2023-12-26 00:00:00-05:00,193.363270,193.642911,192.584265,192.803986,28919300,0.0,0.0
2023-12-27 00:00:00-05:00,192.244692,193.253399,190.846467,192.903839,48087700,0.0,0.0


In [3]:
#Lire les données boursières AAPL
file_path='Data/AAPL.csv'
data=pd.read_csv(file_path,index_col='Date',parse_dates=True)

#date de début et date de fin
start_date='2022-01-03'
end_date='2023-12-29'

data=data.loc[start_date:end_date]

#Initialiser l'indicateur cumulé
total_investment=0
#total_cash_flow=monthly_investment
accumulated_cash_flow=0
monthly_returns=[] #Retours mensuels du return_rate
peak=data['Close'].iloc[0] #Calculer le max drawdown
max_drawdown=0
#Taux sans risque annualisé, sur 252 jours de bourse
risk_free_rate_annual=0.02
risk_free_rate_daily=risk_free_rate_annual/252

#Montant d'investissement mensuel fixe
monthly_investment=100

#Traiter les données par mois
for i,month_end in enumerate(data.resample('ME').last().index):
    #Obtenez des données réelles pour chaque mois
    if month_end.month==1:
        month_start=pd.Timestamp(year=month_end.year-1,month=12,day=1)
    else:
        month_start=pd.Timestamp(year=month_end.year,month=month_end.month-1,day=1)
    monthly_data=data.loc[month_start:month_end,'Close']
    
    if monthly_data.empty:
        continue

    #Obtenez les prix de début et de fin pour chaque mois
    buy_price=monthly_data.iloc[0]
    sell_price=monthly_data.iloc[-1]

    #Calculer les rendements mensuels et les flux de trésorerie
    monthly_return=(sell_price/buy_price)-1
    monthly_returns.append(monthly_return)
    #Investissement cumulé et flux de trésorerie
    monthly_cash_flow=100*(1+monthly_return)
    accumulated_cash_flow += monthly_cash_flow
    total_investment += 100

    #Calculer volatility
    monthly_volatility = np.std(monthly_returns)
    #annualized_volatility = monthly_volatility * np.sqrt(12)
    #Mettre à jour le max drawdown
    monthly_peak=monthly_data.max()
    if monthly_peak>peak:
        peak=monthly_peak
    monthly_drawdown=(peak-monthly_data.min())/peak
    max_drawdown=max(max_drawdown,monthly_drawdown)

    print("Month", month_end.strftime('%Y-%m'), 
      ": Monthly Return:", monthly_return, 
      ", Monthly Cash Flow:", monthly_cash_flow,
      ", Accumulated Cash Flow:", accumulated_cash_flow,
      ", Total Investment:", total_investment)


    
#Calculer le rendement total et le ratio de Sharpe
total_return=(accumulated_cash_flow-total_investment)/total_investment
excess_returns=np.array(monthly_returns)-(risk_free_rate_annual/12)
sharpe_ratio=np.mean(excess_returns)/np.std(excess_returns)*np.sqrt(12)
#Calculer volatility
monthly_volatility = np.std(monthly_returns)
#print de résultats
#print de résultats
print("Total Investment:",total_investment)
print("Total Cash Flow:",accumulated_cash_flow)
print("Total Return (%):",total_return*100)
print("Ratio de Sharpe:",sharpe_ratio)
print("Max Drawdown (%):",max_drawdown*100)
print("Volatility(%):",monthly_volatility*100)

Month 2022-01 : Monthly Return: -0.03972306986288898 , Monthly Cash Flow: 96.0276930137111 , Accumulated Cash Flow: 96.0276930137111 , Total Investment: 100
Month 2022-02 : Monthly Return: -0.09279709850776419 , Monthly Cash Flow: 90.72029014922359 , Accumulated Cash Flow: 186.7479831629347 , Total Investment: 200
Month 2022-03 : Monthly Return: 0.0 , Monthly Cash Flow: 100.0 , Accumulated Cash Flow: 286.7479831629347 , Total Investment: 300
Month 2022-04 : Monthly Return: -0.03400737227659356 , Monthly Cash Flow: 96.59926277234064 , Accumulated Cash Flow: 383.34724593527534 , Total Investment: 400
Month 2022-05 : Monthly Return: -0.1461189924699613 , Monthly Cash Flow: 85.38810075300387 , Accumulated Cash Flow: 468.7353466882792 , Total Investment: 500
Month 2022-06 : Monthly Return: -0.13446445043294253 , Monthly Cash Flow: 86.55355495670575 , Accumulated Cash Flow: 555.288901644985 , Total Investment: 600
Month 2022-07 : Monthly Return: 0.09279797706903015 , Monthly Cash Flow: 109.2

In [4]:
#Ajouter les tickers du portefeuille à une liste appelée 'tickers'
tickers=['AAPL','AMZN','TSLA','GOOGL','PFE']
start_date='2022-01-03'
end_date='2023-12-29'
    
#Réorganiser cette liste par ordre alphabétique
Tickers=sorted(tickers)

#Crée une dataframe vide pour notre portefeuille
pf_cours=pd.DataFrame()

#boucle pour stocker les prix journaliers de clôture
for tickers in Tickers:
    stock=yf.Ticker(tickers)
    cours=stock.history(start=start_date,end=end_date)
    pf_cours[tickers]=cours['Close']
print("pf_cours:")
print(pf_cours)


pf_cours:
                                 AAPL        AMZN       GOOGL        PFE  \
Date                                                                       
2022-01-03 00:00:00-05:00  179.724548  170.404495  144.991501  51.708519   
2022-01-04 00:00:00-05:00  177.443558  167.522003  144.399506  49.773438   
2022-01-05 00:00:00-05:00  172.723572  164.356995  137.774994  50.777493   
2022-01-06 00:00:00-05:00  169.840256  163.253998  137.747498  50.056396   
2022-01-07 00:00:00-05:00  170.008133  162.554001  137.016998  50.859634   
...                               ...         ...         ...        ...   
2023-12-21 00:00:00-05:00  194.431885  153.839996  140.419998  27.890295   
2023-12-22 00:00:00-05:00  193.353287  153.419998  141.490005  27.978962   
2023-12-26 00:00:00-05:00  192.803986  153.410004  141.520004  27.988813   
2023-12-27 00:00:00-05:00  192.903839  153.339996  140.369995  28.185850   
2023-12-28 00:00:00-05:00  193.333298  153.380005  140.229996  28.363180   

 

In [5]:
#les rendements
daily_returns=pf_cours.pct_change()
print("daily_returns:")
print(daily_returns)

daily_returns:
                               AAPL      AMZN     GOOGL       PFE      TSLA
Date                                                                       
2022-01-03 00:00:00-05:00       NaN       NaN       NaN       NaN       NaN
2022-01-04 00:00:00-05:00 -0.012692 -0.016916 -0.004083 -0.037423 -0.041833
2022-01-05 00:00:00-05:00 -0.026600 -0.018893 -0.045876  0.020173 -0.053471
2022-01-06 00:00:00-05:00 -0.016693 -0.006711 -0.000200 -0.014201 -0.021523
2022-01-07 00:00:00-05:00  0.000988 -0.004288 -0.005303  0.016047 -0.035447
...                             ...       ...       ...       ...       ...
2023-12-21 00:00:00-05:00 -0.000770  0.011307  0.015035  0.024611  0.029781
2023-12-22 00:00:00-05:00 -0.005547 -0.002730  0.007620  0.003179 -0.007701
2023-12-26 00:00:00-05:00 -0.002841 -0.000065  0.000212  0.000352  0.016116
2023-12-27 00:00:00-05:00  0.000518 -0.000456 -0.008126  0.007040  0.018822
2023-12-28 00:00:00-05:00  0.002226  0.000261 -0.000997  0.006291 -0.0315

In [6]:
#les rendements moyens
rendement_moyen=daily_returns.mean()
#les rendements annuels
stock_symbols=['AAPL','AMZN','TSLA','GOOGL','PFE']
data_folder='Data'

for symbol in stock_symbols:
    file_path=os.path.join(data_folder,f'{symbol}.csv')
    stock_data=pd.read_csv(file_path,index_col='Date',parse_dates=True)
stock_data={}

jours_ouvres=AAPL_cours['Close'].count()
rendement_annuel=rendement_moyen*jours_ouvres
#la matrice de variance-covariance
pf_var_cov=daily_returns.cov()
print("pf_var_cov:")
print(pf_var_cov)

pf_var_cov:
           AAPL      AMZN     GOOGL       PFE      TSLA
AAPL   0.000336  0.000306  0.000284  0.000074  0.000402
AMZN   0.000306  0.000721  0.000402  0.000082  0.000516
GOOGL  0.000284  0.000402  0.000484  0.000075  0.000393
PFE    0.000074  0.000082  0.000075  0.000247  0.000069
TSLA   0.000402  0.000516  0.000393  0.000069  0.001439


In [7]:
start_date = '2022-01-01'
end_date = '2023-12-29'
date_range = pd.date_range(start_date, end_date, freq='MS')

def get_optimal_weights_for_month(pf_var_cov, rendement_annuel, jours_ouvres, Tickers, tx_sans_risque=0.02):
    num_portfolios = 100000
    best_sharpe_ratio = -np.inf
    best_weights = np.zeros(len(Tickers))

    for i in range(num_portfolios):
        weights = np.random.random(len(Tickers))
        weights /= np.sum(weights)
        rendement_moyen_annualisé = np.dot(weights, rendement_annuel)
        volatilite = np.sqrt(np.dot(weights.T, np.dot(pf_var_cov, weights)))
        volatilite_moyen_annualisee = volatilite * np.sqrt(jours_ouvres)
        sharpe_ratio = (rendement_moyen_annualisé - tx_sans_risque) / volatilite_moyen_annualisee

        if sharpe_ratio > best_sharpe_ratio:
            best_sharpe_ratio = sharpe_ratio
            best_weights = weights
            
    return best_weights

# 初始化存储每月最优权重的字典
monthly_optimal_weights = {}

start_date = pd.to_datetime(start_date)  # 确保 start_date 是 datetime 类型
# 调整 start_date 为该月的第一天
start_date_adjusted = start_date.replace(day=1)
# 遍历每个月，获取每月最优权重
for month_start in pd.date_range(start_date, end_date, freq='MS'):
    # 假设这里是根据实际情况计算的rendement_annuel和pf_var_cov
    optimal_weights = get_optimal_weights_for_month(pf_var_cov, rendement_annuel, jours_ouvres, Tickers)
    monthly_optimal_weights[month_start.strftime('%Y-%m')] = optimal_weights

# 打印每个月的最优权重
for month, weights in monthly_optimal_weights.items():
    print(f"{month}: {weights}")  

2022-01: [0.78792828 0.09815449 0.09599161 0.00144931 0.01647631]
2022-02: [0.92094497 0.01691845 0.02657785 0.01134317 0.02421556]
2022-03: [0.71975328 0.06095318 0.20639478 0.00140674 0.01149203]
2022-04: [0.94339246 0.03243819 0.00650708 0.00612412 0.01153815]
2022-05: [0.56724338 0.07899561 0.3429016  0.00585958 0.00499983]
2022-06: [0.79786318 0.1103859  0.07383473 0.01039903 0.00751716]
2022-07: [6.99904657e-01 2.63006909e-02 2.58795493e-01 1.46533168e-02
 3.45842532e-04]
2022-08: [0.92132318 0.01792464 0.02124481 0.03127642 0.00823095]
2022-09: [0.69183694 0.00762886 0.28406592 0.00823118 0.0082371 ]
2022-10: [8.83227357e-01 4.63082696e-02 3.81523377e-02 4.33644680e-04
 3.18783908e-02]
2022-11: [0.64205196 0.09201178 0.2468592  0.00717144 0.01190562]
2022-12: [8.65092069e-01 1.07460779e-01 2.14806307e-02 1.58637470e-04
 5.80788361e-03]
2023-01: [0.853826   0.0341791  0.06391033 0.03664468 0.01143989]
2023-02: [8.01366871e-01 8.07815180e-02 1.00395900e-01 1.68686942e-02
 5.870175

In [20]:

# 将日收益率转换为月收益率
monthly_returns = daily_returns.resample('ME').apply(lambda x: (x + 1).prod() - 1)
print("monthly_returns:")
print(monthly_returns)
# 初始化累积现金流变量
accumulated_cash_flow = 0
monthly_investment=0
# 初始化存储每月指标的DataFrame
monthly_metrics = pd.DataFrame(index=monthly_returns.index, columns=['return', 'cash_flow', 'sharpe_ratio', 'volatility', 'accumulated_cash_flow'])

# 遍历每个月，计算并更新指标
for month in monthly_returns.index:
    month_str = month.strftime('%Y-%m')
    weights = monthly_optimal_weights.get(month_str)
    if weights is not None:
        # 计算加权月度收益
        weighted_returns = (monthly_returns.loc[month] * weights).sum()
        #cash flow
        cash_flow=100*(1+weighted_returns)
        # 更新累积现金流
        accumulated_cash_flow += cash_flow
        # 计算累积投资额
        monthly_investment += 100
        total_investment = monthly_investment
        #ratio de sharpe
        excess_returns=np.array(monthly_returns)-(risk_free_rate_annual/12)
        sharpe_ratio=np.mean(excess_returns)/np.std(excess_returns)*np.sqrt(12)
        #volatility
        volatility = np.sqrt(np.dot(optimal_weights, np.dot(pf_var_cov, optimal_weights)))
        # 更新DataFrame
        monthly_metrics.loc[month, 'return'] = weighted_returns
        monthly_metrics.loc[month, 'cash_flow'] = cash_flow
        monthly_metrics.loc[month, 'accumulated_cash_flow'] = accumulated_cash_flow
        monthly_metrics.loc[month, 'total_investment'] = total_investment

        monthly_metrics.loc[month, 'sharpe_ratio'] = sharpe_ratio
        monthly_metrics.loc[month, 'volatility'] =volatility

# 打印结果
print("Monthly Metrics:")
print(monthly_metrics)
print("Ratio de Sharpe:",sharpe_ratio)
#print("Max Drawdown (%):",max_drawdown*100)
print("Volatility(%):",volatility*100)



monthly_returns:
                               AAPL      AMZN     GOOGL       PFE      TSLA
Date                                                                       
2022-01-31 00:00:00-05:00 -0.039723 -0.122244 -0.066818 -0.062831 -0.219257
2022-02-28 00:00:00-05:00 -0.054066  0.026673 -0.001822 -0.109129 -0.070768
2022-03-31 00:00:00-04:00  0.057473  0.061437  0.029695  0.102898  0.238009
2022-04-30 00:00:00-04:00 -0.097131 -0.237525 -0.179467 -0.052154 -0.191945
2022-05-31 00:00:00-04:00 -0.054496 -0.032764 -0.003045  0.089719 -0.129197
2022-06-30 00:00:00-04:00 -0.081430 -0.116459 -0.042185 -0.011501 -0.111888
2022-07-31 00:00:00-04:00  0.188634  0.270596  0.067518 -0.029145  0.323765
2022-08-31 00:00:00-04:00 -0.031208 -0.060615 -0.069635 -0.104534 -0.072489
2022-09-30 00:00:00-04:00 -0.120977 -0.108622 -0.116152 -0.032501 -0.037589
2022-10-31 00:00:00-04:00  0.109551 -0.093451 -0.011918  0.063757 -0.142168
2022-11-30 00:00:00-05:00 -0.033027 -0.057595  0.068564  0.086137 -0.14